In [2]:
# Import Required Python Base Packages
import sys
import unittest
import numpy as np
import random
import collections
import os
import time as tp
import glob

# Create a Safety Mechanism for Not Having MatPlotLib Installed
try:
    %matplotlib inline
    import matplotlib.pyplot as plt
    from matplotlib.pyplot import *
    from mpl_toolkits.mplot3d import Axes3D
    HAS_MATPLOTLIB = True
    
    matplotlib.rcParams['figure.figsize'] = (16, 6)
    matplotlib.rcParams['font.size'] = (14)
except ImportError:
    HAS_MATPLOTLIB = False

# Import the Amuse Base Packages
from amuse import datamodel
from amuse.units import nbody_system
from amuse.units import units
from amuse.units import constants
from amuse.datamodel import particle_attributes
from amuse.io import *
from amuse.lab import *

# Import the Amuse Stellar Packages
from amuse.ic.kingmodel import new_king_model
from amuse.ic.salpeter import new_salpeter_mass_distribution_nbody

# Import the Amuse Gravity & Close-Encounter Packages
from amuse.community.ph4.interface import ph4 as grav
from amuse.community.smalln.interface import SmallN
from amuse.community.kepler.interface import Kepler
from amuse.couple import multiples

# Importing cPickle/Pickle
try:
   import cPickle as pickle
except:
   import pickle

from tycho import read, util

In [3]:
set1 = read_set_from_file("/home/draco/jthornton/stars.out", format='hdf5')    

In [4]:
print set1[0].age

2.90798936307e+13 s


In [5]:
# Import Encounter file and open Kepler
file_prefix = "Josh2"
file_directory = "/home/draco/jthornton/Tycho/Encounters/"
file_end = "_encounters.pkl"
encounterInfoReload = None
encounter_file = open(file_directory+file_prefix+file_end, "rb")
encounterInfoReload = pickle.load(encounter_file)
encounter_file.close()

In [9]:
def get_component_binary_elements(comp1, comp2, kep, peri = 0):

    mass = comp1.mass + comp2.mass
    pos = comp2.position - comp1.position
    vel = comp2.velocity - comp1.velocity
    kep.initialize_from_dyn(mass, pos[0], pos[1], pos[2],
                            vel[0], vel[1], vel[2])
    a,e = kep.get_elements()
    r = kep.get_separation()
    E,J = kep.get_integrals()           # per unit reduced mass, note
    if peri:
        M,th = kep.get_angles()
        if M < 0:
            kep.advance_to_periastron()
        else:
            kep.return_to_periastron()
    t = kep.get_time()

    return mass,a,e,r,E,t

In [31]:
from collections import defaultdict

encounterInformation = defaultdict(list)
for key in encounterInfoReload:
    encounterInformation[key] = []
Mass = []
semimajor = []
eccentricity = []
Energy = []
Time = []
master_set, ic_array, converter = read.read_initial_state(file_prefix)
kep = Kepler(unit_converter=converter, redirection="none")
kep.initialize_code()

In [33]:
# Currently outputting 0 for all times in the oneset I've tried

# For loop to go through all keys in the dictionary
for key in encounterInfoReload:
    
    # try loop to cover for empty spaces in the dictionary for shorter runs
    try:
        i=0
        while i < len(encounterInfoReload[key])-2:
            
            # Method from Tyler's code
            mass,a,e,r,E,t = get_component_binary_elements(encounterInfoReload[key][i][0],encounterInfoReload[key][i+1][1],kep)
            # i +2 here to loop through the dictionary correctly (function takes 2 elements at a time)
            i+=2
            
            # Store the information in lists in the loop then append afterwards
            Mass.append(mass)
            semimajor.append(a)
            eccentricity.append(e)
            Energy.append(E)
            Time.append(t)
            
        # Store information in dictionary
        encounterInformation[key].append(Mass)
        encounterInformation[key].append(semimajor)
        encounterInformation[key].append(eccentricity)
        encounterInformation[key].append(Energy)
        encounterInformation[key].append(Time)
        
        # Clean the lists
        Mass = []
        semimajor = []
        eccentricity = []
        Energy = []
        Time = []
        
    except:
        pass
    